## Target configuration

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()#level=logging.WARNING)

2018-01-15 15:01:17,270 INFO    : root         : Using LISA logging configuration:
2018-01-15 15:01:17,273 INFO    : root         :   /home/leoy/Work/tools/lisa-ml-20171106/logging.conf


In [2]:
%matplotlib inline

import os
import json

import devlib
from env import TestEnv

import trappy
# Support for trace events analysis
from trace import Trace

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from wlgen import RTA, Periodic, Ramp

In [3]:
res_dir = '/home/leoy/Work/tools/lisa-ml-20171106/results/wltests/nrg_calc_duration_LL/wa.6d21dcf_android-hikey-linaro-4.9-opt-nrg-cal-task-basis/wk1-homescreen-1'
trace_file = os.path.join(res_dir, "trace.dat")

In [4]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'android',
    "board"        : 'hikey960',
    
    # Device serial ID
    # Not required if there is only one device connected to your computer
    "device"       : "270182BA020B1AA2",
    
    "ANDROID_HOME" : "/home/leoy/Work/tools/lisa-mainline/tools/android-sdk-linux",
    
    # Energy Meters Configuration for BayLibre's ACME Cape
    "emeter" : {
        "instrument" : "aep",
        "conf" : {
            'resistor_values' : [0.033],
            'device_entry'    : '/dev/ttyACM0'
        },
        "channel_map" : {
            "Device0" : "BAT"
        }
    },

    # Folder where all the results will be collected
    "results_dir" : "taskset_test",

    # Define devlib modules to load
    "modules"     : [
        'cpufreq',
        'cpuidle'
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_sugov_util",
            "sched_load_avg_cpu",
            "sched_boost_cpu",
            "cpu_frequency",
             "sched_energy_calc_duration"
         ],
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'taskset', 'rt-app', 'sysbench'],
}

In [5]:
'''
Define platform parameters, these parameter is coming from
file platform.json; but we platform.json file is generated
after one running with LISA on the board. So for analysis
if we have no platform.json file we can define the variable
for this.
'''

platform = {
    'kernel': {
        'major': 4,
        'sha1': '78e8b90',
        'version': '57 SMP PREEMPT Sat Jul 1 15:42:00 CST 2017',
        'version_number': 4,
        'parts': [4, 4, 74],
        'rc': None,
        'release': '4.4.74-g78e8b90',
        'minor': 74
    },
    
    'freqs': {
        'big': [
            903000,
            1421000,
            1805000,
            2112000,
            2362000
        ],
        'little': [
            533000,
            999000,
            1402000,
            1709000,
            1844000]
    },
    
    'nrg_model': {
        'big': {
            'cluster': {
                'nrg_max': 559
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 4269
            }
        },
        'little': {
            'cluster': {
                'nrg_max': 181
            },
            'cpu': {
                'cap_max': 466,
                'nrg_max': 731
            }
        }
    },
    
    'clusters': {
        'big': [4, 5, 6, 7],
        'little': [0, 1, 2, 3]
    },
    'cpus_count': 8,
    'topology': [[0, 1, 2, 3], [4, 5, 6, 7]]
}

In [6]:
trace = Trace(platform, trace_file, events=my_conf['ftrace']['events'], normalize_time=True, window=(5, 25))

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.2.2 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
2018-01-15 15:01:25,819 WARNING : Trace        : Failed to load tasks names from trace events
2018-01-15 15:01:25,825 INFO    : Trace        : Platform clusters verified to be Frequency coherent


In [7]:
# Analyse CPU utilization standard devation
def analysis_energy_calculation_duration():

    df = trace.data_frame.trace_event('sched_energy_calc_duration')
    
    if df.empty:
        return
    
    calc_duration_df = pd.DataFrame(
        {
            'max': [int(df['duration'].max())],
            'min': [int(df['duration'].min())],
            'mean': [int(df['duration'].mean())],
        },
    )
    
    return calc_duration_df

In [8]:
analysis_energy_calculation_duration()

,max,mean,min
0,48438,21093,12500
